<a href="https://colab.research.google.com/github/seyhah/ETC3550Slides/blob/master/tf_cleandata_streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 82.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.5 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.preprocessing import LabelEncoder
import datetime
import nltk
from nltk.corpus import stopwords

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Download the stopwords from NLTK
nltk.download('stopwords')

# Get the list of stop words
stop_words = set(stopwords.words('english'))

# Define a function to clean the text
def clean_text(text):
    text = str(text)  # Convert to string if not already
    text = text.lower()  # Lowercase the text
    text = text.strip()  # Remove leading and trailing whitespaces
    text = ''.join(char for char in text if char.isalnum() or char.isspace())  # Remove non-alphanumeric characters
    text = ' '.join(word for word in text.split() if word not in stop_words)  # Remove stop words
    return text

# Load data from Excel file (assuming it's uploaded to Google Drive)
df = pd.read_excel("/content/drive/MyDrive/combined_output.xlsx", usecols=['code', 'measure'])

# Drop rows with missing values in the 'measure' and 'code' columns
df = df.dropna(subset=["measure", "code"])

# Clean text data
df['measure'] = df['measure'].apply(clean_text)

# Prepare label encoding
le = LabelEncoder()
df['label'] = le.fit_transform(df['code'])
# Save label encoder
le_path = '/content/drive/My Drive/label_encoder.pkl'
with open(le_path, 'wb') as f:
    pickle.dump(le, f)

print(f"Label Encoder saved to {le_path}")

# Split data into train, validation, and test sets
train, val, test = np.split(df.sample(frac=1), [int(0.8 * len(df)), int(0.9 * len(df))])

# Function to create a dataset from a dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=1024):
    df = dataframe.copy()
    labels = df.pop('label')
    ds = tf.data.Dataset.from_tensor_slices((df['measure'].values, labels.values))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    return ds

# Prepare datasets
train_data = df_to_dataset(train)
valid_data = df_to_dataset(val, shuffle=False)
test_data = df_to_dataset(test, shuffle=False)

# Build and compile the model
embedding_url = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding_url, input_shape=[], dtype=tf.string, trainable=True)
model = tf.keras.Sequential([
    hub_layer,
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(len(le.classes_), activation='softmax')
])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Label Encoder saved to /content/drive/My Drive/label_encoder.pkl


In [ ]:
# Load the trained model and label encoder from Google Drive
#model_path = '/content/drive/MyDrive/text_classification_model_20240613_100558.h5'
#model = tf.keras.models.load_model(model_path, custom_objects={'KerasLayer': hub.KerasLayer})

In [ ]:
# Train the model
#model.fit(train_data, epochs=10, validation_data=valid_data)

# Save the trained model to Google Drive
#timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
#model_path = f'/content/drive/My Drive/text_classification_model_{timestamp}.h5'
#model.save(model_path)

#print(f"Model saved to {model_path}")

Epoch 1/10
70/70 [==============================] - 103s 1s/step - loss: 1.0100 - accuracy: 0.6755 - val_loss: 1.1714 - val_accuracy: 0.6603
Epoch 2/10
70/70 [==============================] - 105s 1s/step - loss: 0.9939 - accuracy: 0.6785 - val_loss: 1.1905 - val_accuracy: 0.6611
Epoch 3/10
70/70 [==============================] - 102s 1s/step - loss: 0.9860 - accuracy: 0.6826 - val_loss: 1.1990 - val_accuracy: 0.6586
Epoch 4/10
70/70 [==============================] - 105s 1s/step - loss: 0.9766 - accuracy: 0.6881 - val_loss: 1.2134 - val_accuracy: 0.6544
Epoch 5/10
70/70 [==============================] - 110s 2s/step - loss: 0.9635 - accuracy: 0.6894 - val_loss: 1.2334 - val_accuracy: 0.6526
Epoch 6/10
70/70 [==============================] - 105s 1s/step - loss: 0.9559 - accuracy: 0.6913 - val_loss: 1.2480 - val_accuracy: 0.6539
Epoch 7/10
70/70 [==============================] - 106s 2s/step - loss: 0.9447 - accuracy: 0.6925 - val_loss: 1.2736 - val_accuracy: 0.6559
Epoch 8/10
70

In [ ]:
import datetime
from tensorflow.keras.callbacks import ModelCheckpoint

# Train the model
history = model.fit(
    train_data,
    epochs=5,
    validation_data=valid_data
)

# Get the final training and validation accuracy
train_accuracy = history.history['accuracy'][-1]
val_accuracy = history.history['val_accuracy'][-1]

# Format the accuracy values to two decimal places
train_accuracy = f"{train_accuracy:.2f}"
val_accuracy = f"{val_accuracy:.2f}"

# Save the trained model to Google Drive with accuracy in the filename
timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
model_path = f'/content/drive/My Drive/text_classification_model_trainAcc{train_accuracy}_valAcc{val_accuracy}_{timestamp}.h5'
model.save(model_path)


Epoch 1/5
70/70 [==============================] - 107s 2s/step - loss: 0.8722 - accuracy: 0.7109 - val_loss: 0.9703 - val_accuracy: 0.6946
Epoch 2/5
70/70 [==============================] - 107s 2s/step - loss: 0.8646 - accuracy: 0.7137 - val_loss: 0.9804 - val_accuracy: 0.6947
Epoch 3/5


KeyboardInterrupt: 

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd
import pickle
from google.colab import drive
import nltk
from nltk.corpus import stopwords

# Mount Google Drive
drive.mount('/content/drive')

# Download the stopwords from NLTK
nltk.download('stopwords')

# Get the list of stop words
stop_words = set(stopwords.words('english'))

# Function to clean the text
def clean_text(text):
    text = str(text).lower().strip()  # Convert to string, lowercase, and strip
    text = ''.join(char for char in text if char.isalnum() or char.isspace())  # Remove non-alphanumeric characters
    text = ' '.join(word for word in text.split() if word not in stop_words)  # Remove stop words
    return text


# Load label encoder
le_path = '/content/drive/MyDrive/label_encoder.pkl'
with open(le_path, 'rb') as f:
    le = pickle.load(f)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Load the trained model and label encoder from Google Drive
model_path = '/content/drive/MyDrive/text_classification_model_trainAcc0.70_valAcc0.65_20240613_133404.h5'
#model_path = '/content/drive/MyDrive/text_classification_model_trainAcc0.71_valAcc0.70_20240615_001124.h5'
model = tf.keras.models.load_model(model_path, custom_objects={'KerasLayer': hub.KerasLayer})

In [ ]:
# Function to predict the top 3 codes for a single measure
def predict_top_3_codes(model, measure):
    measure = clean_text(measure)  # Clean the input measure text
    prediction = model.predict([measure])  # Make prediction
    top_3_indices = np.argsort(prediction, axis=1)[:, -5:][:, ::-1]  # Get the top 3 indices
    top_3_codes = le.inverse_transform(top_3_indices[0])  # Convert indices to original codes
    return top_3_codes  # Return the top 3 predicted codes
# Example usage for single measure prediction
single_measure = "All plant products, plant products and other items subject to phytosanitary inspection must be properly packaged to ensure that they are not torn open and stored in a container that does not leak during transport."
predicted_codes = predict_top_3_codes(model, single_measure)
print(f"Top 3 predicted codes for the measure: {predicted_codes}")

1/1 [==============================] - 0s 42ms/step
Top 3 predicted codes for the measure: ['B42' 'B33' 'A64' 'P6' 'A33']


In [ ]:
# This is the streamlit app
import streamlit as st
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.preprocessing import LabelEncoder
import pickle

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Function to clean text
def clean_text(text):
    text = str(text).lower().strip()
    text = ''.join(char for char in text if char.isalnum() or char.isspace())
    return text

# Function to predict a single measure
def predict_single_measure(model, measure, label_encoder):
    measure = clean_text(measure)
    prediction = model.predict([measure])
    predicted_label = np.argmax(prediction, axis=1)
    predicted_code = label_encoder.inverse_transform(predicted_label)
    return predicted_code[0]

# Function to predict top 3 codes for batch measures
def predict_top_3(model, measures, label_encoder):
    cleaned_measures = [clean_text(m) for m in measures]
    predictions = model.predict(cleaned_measures)
    top_3_indices = np.argsort(predictions, axis=1)[:, -3:][:, ::-1]
    top_3_codes = label_encoder.inverse_transform(top_3_indices)
    return top_3_codes

# Load the trained model from Google Drive
model_path1 = '/content/drive/MyDrive/text_classification_model_20240613_074451.h5'
model = tf.keras.models.load_model(model_path1, custom_objects={'KerasLayer': hub.KerasLayer})

# Load label encoder
le_path = '/content/drive/My Drive/label_encoder.pkl'
with open(le_path, 'rb') as f:
    le = pickle.load(f)

# Streamlit app
def main():
    st.title("Text Classification App")

    # Single measure prediction
    st.header("Make Single Measure Prediction")
    measure_input = st.text_input("Enter a measure for prediction:")
    if st.button("Predict"):
        if measure_input:
            predicted_code = predict_single_measure(model, measure_input, le)
            st.write(f"Predicted code for the measure: {predicted_code}")

    # Batch prediction
    st.header("Batch Prediction")
    uploaded_file = st.file_uploader("Upload an Excel file for batch prediction:", type="xlsx")
    if uploaded_file is not None:
        input_df = pd.read_excel(uploaded_file)
        if 'measure' in input_df.columns:
            batch_measures = input_df['measure'].tolist()
            predicted_codes = predict_top_3(model, batch_measures, le)
            input_df['predicted_code_1'] = predicted_codes[:, 0]
            input_df['predicted_code_2'] = predicted_codes[:, 1]
            input_df['predicted_code_3'] = predicted_codes[:, 2]
            st.write(input_df)
        else:
            st.error("Input file must contain 'measure' column.")

if __name__ == "__main__":
    main()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


2024-06-13 09:16:32.793 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-06-13 09:16:32.795 Session state does not function when running a script without `streamlit run`


In [ ]:
! wget -q -O - ipv4.icanhazip.com

34.85.252.139


In [ ]:
! streamlit run /content/drive/MyDrive/app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.85.252.139:8501

npx: installed 22 in 2.364s
your url is: https://rotten-waves-tease.loca.lt
2024-06-13 09:13:46.850696: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-13 09:13:46.850762: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-13 09:13:46.852598: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-13 09:13:46.864218: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimi

In [ ]:
!streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.85.252.139:8501

  Stopping...
  Stopping...
